In [ ]:
!pip install spacy
!python -m spacy download ru_core_news_sm
!pip install sentence-transformers
!pip install pinecone

### Config things

In [101]:
import os

# Получаем текущий рабочий каталог
current_dir = os.getcwd()

# Строим путь на три уровня вверх
config_path = os.path.join(current_dir, '..', '..', 'llama.config')

# Нормализуем путь (убираем лишние '..' и т.д.)
config_path = os.path.normpath(config_path)


# Читаем файл
with open(config_path, 'r') as file:
    api_base = file.readline().strip()
    my_model = file.readline().strip()
    API_KEY = file.readline().strip()
    YOUR_API_KEY = file.readline().strip()
#    index_name = file.readline().strip()
index_name = 'red-llama-bertopic'

# Выводим значения
#print(f"API Base: {api_base}")
#print(f"My Model: {my_model}")
#print(f"API Key: {API_KEY}")
#print(f"Pinecone API Key: {YOUR_API_KEY}")
#print(f"Pinecone index name: {index_name}")


### Мультиязычная модель. Обучена!

```Python
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from bertopic.representation import KeyBERTInspired

# Используем модель, поддерживающую русский язык
#sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
sentence_model = SentenceTransformer('distiluse-base-multilingual-cased')

# Мои настройки
my_nr_topics = 14 # Количество тем, которые вы хотите извлечь. Можете настроить это значение, чтобы контролировать количество тем.
my_nr_repr_docs=7 # Количество документов, используемых для представления каждой темы
my_min_topic_size=10  # Минимальный размер темы, сколько документов должно быть в теме, чтобы она считалась значимой
му_top_n_words=10 # Количество слов, которые вы хотите получить для каждой темы

# Настройка модели представления
representation_model = KeyBERTInspired(nr_repr_docs=my_nr_repr_docs, top_n_words=му_top_n_words)

# Создание модели BERTopic с настройками
topic_model = BERTopic(
    embedding_model=sentence_model,
    representation_model=representation_model,
    nr_topics=my_nr_topics,
    min_topic_size=my_min_topic_size,  # Минимальный размер темы
    calculate_probabilities=True,  # Вычисление вероятностей
    verbose=True  # Вывод дополнительной информации
)

# Обучение модели
#topics, probs = topic_model.fit_transform(docs)

# Сохранение модели
#topic_model.save("red_bertopic_model")

# Загрузка модели
from bertopic import BERTopic
loaded_model = BERTopic.load("red_bertopic_model")

# Добавление тем и вероятностей в DataFrame
#chunks_df['topic'] = topics
#chunks_df['probs'] = probs


# Вывод информации
print(chunks_df.columns)
print(chunks_df.info())
```

## Загрузка модели в оперативку с Yandex Cloud

In [5]:
# Импорт библиотек
import requests, joblib, io
import pandas as pd
from bertopic import BERTopic

# URL вашей модели
model_url = 'https://storage.yandexcloud.net/cyrilos/red_bertopic_model'

# Загрузка модели
response = requests.get(model_url)

# Проверка успешности запроса
if response.status_code == 200:
    # Загрузка модели из байтового потока
    loaded_model = joblib.load(io.BytesIO(response.content))
    print("Модель успешно загружена в память.")
else:
    print(f"Ошибка загрузки: {response.status_code}")

Модель успешно загружена в память.


## Настройки при обучении

In [7]:
# Мои настройки
my_nr_topics = 14 # Количество тем, которые вы хотите извлечь. Можете настроить это значение, чтобы контролировать количество тем.
my_nr_repr_docs=7 # Количество документов, используемых для представления каждой темы
my_min_topic_size=10  # Минимальный размер темы, сколько документов должно быть в теме, чтобы она считалась значимой
му_top_n_words=10 # Количество слов, которые вы хотите получить для каждой темы


In [16]:
# Получаем информацию по темам
topic_model = loaded_model
print(f"Настройки. Количество тем: {my_nr_topics}, Количество представи. документов: {my_nr_repr_docs}, Мин. размер: {my_min_topic_size}, Количество топ-слов: {му_top_n_words}")
topic_info = topic_model.get_topic_info()
print(topic_info)


Настройки. Количество тем: 14, Количество представи. документов: 7, Мин. размер: 10, Количество топ-слов: 10
   Topic  Count                                        Name  \
0     -1     21  -1_бакалавриат_бакалавр_диплом_аспирантура   
1      0     59     0_французский_бульдог_собака_популярный   
2      1     22     1_red_mad_robot_red_цифровой_разработка   
3      2     54            2_telegram_messages_message_chat   
4      3     62                 3_кожа_кислота_витамин_вода   
5      4     48         4_мост_бетон_строительство_строение   
6      5     19      5_звезда_ресторан_трёхзвёздочный_кухня   
7      6     50   6_работодатель_трудовой_работник_зарплата   
8      7     14              7_class_тренировка_клуб_тренер   
9      8     31       8_платёж_фискальный_оплачивать_оплата   

                                      Representation  \
0  [бакалавриат, бакалавр, диплом, аспирантура, о...   
1  [французский, бульдог, собака, популярный, щен...   
2  [red_mad_robot, red, цифро

In [17]:
# Получаем ключевые слова по темам
print(f"Количество тем заказано: {my_nr_topics}, Количество представи. документов: {my_nr_repr_docs}, Мин. размер: {my_min_topic_size}, Количество топ-слов: {му_top_n_words}")

for id in range(-1, len(topic_info)-1):
    topic_words = topic_model.get_topic(id)
    print(f"Ключевые слова по теме {id}: {topic_words}")


Количество тем заказано: 14, Количество представи. документов: 7, Мин. размер: 10, Количество топ-слов: 10
Ключевые слова по теме -1: [('бакалавриат', np.float32(0.31518373)), ('бакалавр', np.float32(0.27087903)), ('диплом', np.float32(0.2588203)), ('аспирантура', np.float32(0.23891361)), ('образование', np.float32(0.23076478)), ('студент', np.float32(0.19909294)), ('учебный', np.float32(0.18908706)), ('специальность', np.float32(0.18783751)), ('обучение', np.float32(0.18729722)), ('специалитета', np.float32(0.10582714))]
Ключевые слова по теме 0: [('французский', np.float32(0.47135442)), ('бульдог', np.float32(0.15758193)), ('собака', np.float32(0.14411372)), ('популярный', np.float32(0.1424608)), ('щенок', np.float32(0.12345127)), ('порода', np.float32(0.09492709)), ('животное', np.float32(0.0780914)), ('умный', np.float32(0.07168501)), ('дружелюбный', np.float32(0.05522666)), ('компактный', np.float32(0.04894419))]
Ключевые слова по теме 1: [('red_mad_robot', np.float32(0.47562537))

#### **Названия тем:**

- -1: Социальное обеспечение
- 0: Французский бульдог
- 1: rmr (red_mad_robot)
- 2: Telegram
- 3: Уход за кожей
- 4: Мостостроительство
- 5: Michelin
- 6: Трудовой кодекс
- 7: World Class
- 8: Оплата услуг

In [10]:
topics = {
    -1: "Социальное обеспечение",
    0: "Французский бульдог",
    1: "rmr (red_mad_robot)",
    2: "Telegram",
    3: "Уход за кожей",
    4: "Мостостроительство",
    5: "Michelin",
    6: "Трудовой кодекс",
    7: "World Class",
    8: "Оплата услуг"
}

# Чтение словаря в цикле
for key, value in topics.items():
    print(f"{key}: {value}")

-1: Социальное обеспечение
0: Французский бульдог
1: rmr (red_mad_robot)
2: Telegram
3: Уход за кожей
4: Мостостроительство
5: Michelin
6: Трудовой кодекс
7: World Class
8: Оплата услуг


### Поиск по новой новой обученной модели red_bertopic_model

# Поиск в Pinecone c фильтрацией

### Классификация моделью с threshold

In [104]:
import pandas as pd
import numpy as np

# Пример данных для предсказания
docs = [
    "Правила оплаты труда наемного работника",
    "Где лучше заниматься спортом и фитнесом?",
    "Сколько учиться на бакалавриате?"
]

# Использование модели для предсказания тем
topics, probs = loaded_model.transform(docs)

# Создание DataFrame для хранения результатов
docs_df = pd.DataFrame(docs, columns=['document'])

# Установка порога вероятности
threshold = 0.17  # Установите желаемый порог


# Обработка вероятностей и определение тем
if probs.ndim > 1:  # Если probs двумерный
    print(f'Многомерный: {len(probs)}')
    # Создаем DataFrame для вероятностей
    probs_df = pd.DataFrame(probs, columns=[f'Topic_{i}' for i in range(probs.shape[1])])
    
    # Определяем темы на основе порога
    max_probs = probs_df.max(axis=1)  # Находим максимальные вероятности
    topics = np.where(max_probs > threshold, probs_df.idxmax(axis=1).str.replace('Topic_', '').astype(int), -1)
    
    # Объединяем два DataFrame
    docs_df = pd.concat([docs_df, probs_df], axis=1)
else:  # Если probs одномерный
    docs_df['probs'] = probs
    docs_df['topic'] = np.where(probs > threshold, -1, topics)  # Присваиваем -1, если вероятность ниже порога

# Добавляем темы в DataFrame
docs_df['topic'] = topics

# Настройка отображения для вывода всех строк и столбцов
pd.set_option('display.max_rows', None)  # Выводить все строки
pd.set_option('display.max_columns', None)  # Выводить все столбцы

# Вывод информации
print(docs_df)



Batches: 100%|██████████| 1/1 [00:00<00:00, 31.26it/s]

Многомерный: 3
                                   document   Topic_0   Topic_1   Topic_2  \
0   Правила оплаты труда наемного работника  0.011483  0.016343  0.017607   
1  Где лучше заниматься спортом и фитнесом?  0.011754  0.022872  0.023739   
2          Сколько учиться на бакалавриате?  0.000036  0.000224  0.000220   

    Topic_3   Topic_4   Topic_5   Topic_6   Topic_7   Topic_8  topic  
0  0.011563  0.014336  0.032105  0.285219  0.040003  0.044743      6  
1  0.011708  0.014302  0.034047  0.037444  0.291928  0.173945      7  
2  0.000035  0.000044  0.022352  0.020150  0.027005  0.022002     -1  


## Загрузка базы с Pinecone. Функция для запросов

In [105]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone

# Инициализация Pinecone
pc = Pinecone(api_key=YOUR_API_KEY)  # Замените на ваш API-ключ

# Подключение к индексу
index_name = "red-llama-bertopic"
index = pc.Index(index_name)

# Загрузка модели
model = SentenceTransformer('all-MiniLM-L6-v2')

def find_me(doc_n):
    # Индекс строки для анализа
    n = doc_n

    # Проверка, что индекс n находится в пределах DataFrame
    if n >= len(docs_df):
        print(f"Ошибка: индекс {n} выходит за пределы DataFrame.")
        exit()

    # Получение темы документа
    doc_topic = int(docs_df['topic'].iloc[n])

    # Фильтр для поиска в Pinecone
    filter_condition = {
        "topic": {
            "$eq": doc_topic
        }
    }

    # Обработка случая, когда тема не определена
#    if doc_topic == -1:
#        print("Тема не определена (вероятность ниже порога)")
#    else:
    my_topic = 'Topic_' + str(doc_topic)
    if my_topic in docs_df.columns:
        my_prob = docs_df.loc[n, my_topic]
    else:
        my_prob = "Тема не определена (вероятность ниже порога)"


    # Векторизация запроса
    query_text = docs_df['document'].iloc[n]
    query_embedding = model.encode(query_text)

    # Поиск в индексе
    results = index.query(
        vector=query_embedding.tolist(),  # Вектор запроса
        top_k=5,  # Количество возвращаемых результатов
        include_metadata=True,  # Включить метаданные в результаты
        filter=filter_condition  # Условия фильтрации
    )

    # Вывод результатов поиска
    print(f"Текст запроса: {query_text}\nСамая вероятная тема: {doc_topic}: {my_prob}")
    if not results['matches']:
        print("По вашему запросу ничего не найдено.")
    else:
        print("\nРезультаты поиска в Pinecone:")
        for match in results['matches']:
            print(f"ID: {match['id']}, Score: {match['score']}")
            if 'answer' in match['metadata']:
                print(f"Ответ: {match['metadata']['answer']}")
            if 'topic_name' in match['metadata']:
                print(f"Тема: {match['metadata']['topic_name']}")
            print("-" * 100)



### Вывод запросов по документам

In [106]:
find_me(0)

Текст запроса: Правила оплаты труда наемного работника
Самая вероятная тема: 6: 0.2852191746523588

Результаты поиска в Pinecone:
ID: chunk_324, Score: 0.74414736
Ответ: Предоставляются пособие по беременности и родам, единовременное пособие и ежемесячные выплаты по уходу за ребенком.
Тема: Трудовой кодекс
----------------------------------------------------------------------------------------------------
ID: chunk_316, Score: 0.731326461
Ответ: Работнику возмещаются расходы на проезд, проживание, суточные и сохраняется средняя зарплата.
Тема: Трудовой кодекс
----------------------------------------------------------------------------------------------------
ID: chunk_303, Score: 0.720413
Ответ: Работа в выходные и праздничные дни допускается только с согласия работника и оплачивается в двойном размере.
Тема: Трудовой кодекс
----------------------------------------------------------------------------------------------------
ID: chunk_329, Score: 0.718351
Ответ: Работник имеет право на 

In [107]:
find_me(1)

Текст запроса: Где лучше заниматься спортом и фитнесом?
Самая вероятная тема: 7: 0.2919282984408514

Результаты поиска в Pinecone:
ID: chunk_258, Score: 0.481509715
Ответ: Можно и нужно! World Class организует массу разных фитнес-туров как в России, так и за рубежом. Изучайте программы туров здесь: https://camp.worldclass.ru/ и отправляйтесь в фитнес-путешествие с World Class!
Тема: World Class
----------------------------------------------------------------------------------------------------
ID: chunk_260, Score: 0.460304111
Ответ: Вся информация представлена на сайте https://sport.worldclass.ru/. Также информацию о мероприятиях можно получить у тренеров в клубе.
Тема: World Class
----------------------------------------------------------------------------------------------------
ID: chunk_253, Score: 0.436094433
Ответ: В наших клубах есть возможность выбрать формат тренировок, исходя из любых целей и пожеланий. Подбор фитнес-программ по параметрам в клубах World Class на сайте https

In [108]:
find_me(2)

Текст запроса: Сколько учиться на бакалавриате?
Самая вероятная тема: -1: Тема не определена (вероятность ниже порога)

Результаты поиска в Pinecone:
ID: chunk_7, Score: 0.595531106
Ответ: **Как оплатить услуги бонусами от Дом.ру?**
	Участвуйте в программе Дом.ру Бонус. Так на вашем счёте будут копиться бонусы, которыми можно оплатить до 100% стоимости абонентской платы.​ 100 бонусов = 20 ₽. А ещё получайте скидки на услуги наших партнёров.
Тема: Социальное обеспечение
----------------------------------------------------------------------------------------------------
ID: chunk_33, Score: 0.585178196
Ответ: **Какие Минусы специалитета?**
	К минусам специалитета можно отнести более долгий срок обучения. Кроме того, возможность получить специальность есть не во всех вузах.
Тема: Социальное обеспечение
----------------------------------------------------------------------------------------------------
ID: chunk_39, Score: 0.526665151
Ответ: **Что будет с дипломами, полученным до введения 